## McKinsey Investment Office
### Qunatitavive Validation Analyst Test
### -Deeksha Dave

## Part 1

In [1]:
import pandas as pd

In [2]:
## Reading the sheet for first question from the given excel file
df = pd.read_excel('question3.xlsx', sheetname = 'FOR_PROBLEM_1_2')

In [3]:
df.head()

,DATE,TICKER,DAILY_RETURN
0,2010-06-22,AMZN,NaN
1,2010-06-23,AMZN,-0.007031
2,2010-06-24,AMZN,-0.025690
3,2010-06-25,AMZN,0.022564
4,2010-06-28,AMZN,-0.026446


In [4]:
df.set_index('DATE',inplace=True)

In [5]:
## Calculating daily cmulative returns to ease the computation of MTD and YTD returns
df["Cumulative"] = (1+df['DAILY_RETURN']).cumprod()
import warnings
warnings.filterwarnings('ignore')
df['Cumulative'][0] = 1

In [6]:
df.head()

,TICKER,DAILY_RETURN,Cumulative
DATE,,,
2010-06-22,AMZN,NaN,1.000000
2010-06-23,AMZN,-0.007031,0.992969
2010-06-24,AMZN,-0.025690,0.967460
2010-06-25,AMZN,0.022564,0.989290
2010-06-28,AMZN,-0.026446,0.963127


In [7]:
## Grouping the data according to business year, to calculate the YTD return
YTD = df.groupby(pd.TimeGrouper('A'))

In [8]:
## Computing the YTD returns
df["YTD"] = YTD['Cumulative'].transform(lambda x: x/x.iloc[0]-1.0)

In [9]:
## Grouping the data according to business month, to calculate the MTD return
MTD = df.groupby(pd.TimeGrouper('BM'))

In [10]:
## Computing the MTD returns
df['MTD'] = MTD['Cumulative']. transform(lambda x: x/x.iloc[0]-1.0)

In [11]:
df.head()

,TICKER,DAILY_RETURN,Cumulative,YTD,MTD
DATE,,,,,
2010-06-22,AMZN,NaN,1.000000,0.000000,0.000000
2010-06-23,AMZN,-0.007031,0.992969,-0.007031,-0.007031
2010-06-24,AMZN,-0.025690,0.967460,-0.032540,-0.032540
2010-06-25,AMZN,0.022564,0.989290,-0.010710,-0.010710
2010-06-28,AMZN,-0.026446,0.963127,-0.036873,-0.036873


## Part 2

In [12]:
## Only the initial price is given. Hence to calculate the daily close price,I multiplied it with the cumulative returns.
df['Daily Close Price'] = df['Cumulative']*122.31

In [13]:
## Since daily prices were not given, I scraped data from yahoo finance to compute the returns. 
## Using the daily close price only, I calculated the daily returns, MTd and YTD returns.

In [14]:
import pandas_datareader.data as web
import datetime as datetime

# Grab data for 'AMZN' beginning from 22 June 2010 to 21 June 2018
AMZN = web.DataReader('AMZN', 'yahoo', datetime.date(2010,6,22),datetime.date(2018,6,21))

# Again group the data with the same TimeGrouper to get things grouped by year
AMZN_YTD = AMZN.groupby(pd.TimeGrouper('A'))
# Group the data with the same TimeGrouper to get things grouped by business month
AMZN_MTD = AMZN.groupby(pd.TimeGrouper('BM'))

# Create a new column with YTD and MTD data of adjusted close
AMZN['Daily Returns'] = AMZN['Adj Close'].pct_change()
AMZN["YTD"] = AMZN_YTD['Adj Close'].transform(lambda x: x/x.iloc[0]-1.0)
AMZN["MTD"] = AMZN_MTD['Adj Close'].transform(lambda x: x/x.iloc[0]-1.0)

In [15]:
AMZN.head()

,High,Low,Open,Close,Volume,Adj Close,Daily Returns,YTD,MTD
Date,,,,,,,,,
2010-06-22,125.230003,121.550003,122.650002,122.309998,6207200,122.309998,NaN,0.000000,0.000000
2010-06-23,123.220001,120.040001,122.110001,121.449997,5455100,121.449997,-0.007031,-0.007031,-0.007031
2010-06-24,120.849998,116.800003,120.610001,118.330002,7771100,118.330002,-0.025690,-0.032540,-0.032540
2010-06-25,121.760002,117.629997,118.139999,121.000000,5769700,121.000000,0.022564,-0.010710,-0.010710
2010-06-28,120.040001,117.099998,118.849998,117.800003,5612200,117.800003,-0.026446,-0.036873,-0.036873


## Part 3

In [16]:
## Reading the sheet for Question 3
df2 = pd.read_excel('question3.xlsx', sheetname = 'FOR_PROBLEM_3_4')

In [17]:
df2.set_index('DATE', inplace = True)

In [18]:
df2.head()

,TICKER,DAILY_RETURN
DATE,,
2010-06-22,AMZN,NaN
2010-06-23,AMZN,-0.007031
2010-06-24,AMZN,-0.025690
2010-06-25,AMZN,0.022564
2010-06-28,AMZN,-0.026446


In [19]:
## Creating a list of all the unique tickers in dataframe
tickers = df2['TICKER'].unique()

In [20]:
## Groupig the entire dataframe by tickers
trial = df2.groupby('TICKER')

In [21]:
## Calling groups from the grouped dataframe to create seperate dataframes for each ticker
## Using global variable to create multiple dataferames
gbl=globals()
for i in tickers:
    gbl['df_'+i] = trial.get_group(i)
    gbl['df_'+i]['Cumulative'] =  (1+gbl['df_'+i]['DAILY_RETURN']).cumprod()
    gbl['df_'+i]['Cumulative'][0] = 1

In [22]:
## Calculating YTD returns for each ticker in dataset
for i in tickers:    
    gbl['GroupedDat_YTD_'+i] = gbl['df_'+i].groupby(pd.TimeGrouper('A'))
    gbl['df_'+i]['YTD'] = gbl['GroupedDat_YTD_'+i]['Cumulative'].transform(lambda x: x/x.iloc[0]-1.0)  

In [23]:
## Calculating MTD returns for each ticker in dataset
for i in tickers:
    gbl['GroupedDat_MTD_'+i] = gbl['df_'+i].groupby(pd.TimeGrouper('BM'))
    gbl['df_'+i]['MTD'] = gbl['GroupedDat_MTD_'+i]['Cumulative'].transform(lambda x: x/x.iloc[0]-1.0)

In [24]:
## Finding the daily close price using inital prices.
## If daily prices were given, we could perform similar process(Scrape data from web) as in Question 2 to obtain daily, MTD and YTD returns for each ticker.
df_AMZN['Daily Close Price'] = df_AMZN['Cumulative']*122.31
df_GOOG['Daily Close Price'] = df_GOOG['Cumulative']*241.5539
df_AAPL['Daily Close Price'] = df_AAPL['Cumulative']*26.38178

In [25]:
df_AMZN.head()

,TICKER,DAILY_RETURN,Cumulative,YTD,MTD,Daily Close Price
DATE,,,,,,
2010-06-22,AMZN,NaN,1.000000,0.000000,0.000000,122.310000
2010-06-23,AMZN,-0.007031,0.992969,-0.007031,-0.007031,121.449999
2010-06-24,AMZN,-0.025690,0.967460,-0.032540,-0.032540,118.330004
2010-06-25,AMZN,0.022564,0.989290,-0.010710,-0.010710,121.000002
2010-06-28,AMZN,-0.026446,0.963127,-0.036873,-0.036873,117.800005


## Part 4

In [26]:
## Preparing seperate dataframes to merge

In [27]:
df_AMZN = df_AMZN.add_prefix('AMZN_')
df_GOOG = df_GOOG.add_prefix('GOOG_')
df_AAPL = df_AAPL.add_prefix('AAPL_')

In [28]:
portfolio = pd.concat([df_AMZN,df_GOOG,df_AAPL],axis = 1)

In [29]:
## Calculating the cumulative return of the entire equal-weighted portfolio
portfolio['Portfolio_Cumulative'] = ((portfolio['AMZN_Cumulative']) + (portfolio['GOOG_Cumulative']) + (portfolio['AAPL_Cumulative']))/3

In [30]:
## Computing the daily returns of the portfolio
portfolio['Portfolio_Daily_returns'] = portfolio['Portfolio_Cumulative'].pct_change()

In [31]:
portfolio.head()

,AMZN_TICKER,AMZN_DAILY_RETURN,AMZN_Cumulative,AMZN_YTD,AMZN_MTD,AMZN_Daily Close Price,GOOG_TICKER,GOOG_DAILY_RETURN,GOOG_Cumulative,GOOG_YTD,GOOG_MTD,GOOG_Daily Close Price,AAPL_TICKER,AAPL_DAILY_RETURN,AAPL_Cumulative,AAPL_YTD,AAPL_MTD,AAPL_Daily Close Price,Portfolio_Cumulative,Portfolio_Daily_returns
DATE,,,,,,,,,,,,,,,,,,,,
2010-06-22,AMZN,NaN,1.000000,0.000000,0.000000,122.310000,GOOG,NaN,1.000000,0.000000,0.000000,241.553900,AAPL,NaN,1.000000,0.000000,0.000000,26.381780,1.000000,NaN
2010-06-23,AMZN,-0.007031,0.992969,-0.007031,-0.007031,121.449999,GOOG,-0.008638,0.991362,-0.008638,-0.008638,239.467474,AAPL,-0.010517,0.989483,-0.010517,-0.010517,26.104323,0.991271,-0.008729
2010-06-24,AMZN,-0.025690,0.967460,-0.032540,-0.032540,118.330004,GOOG,-0.014418,0.977069,-0.022931,-0.022931,236.014930,AAPL,-0.007270,0.982289,-0.017711,-0.017711,25.914540,0.975606,-0.015803
2010-06-25,AMZN,0.022564,0.989290,-0.010710,-0.010710,121.000002,GOOG,-0.005094,0.972093,-0.027907,-0.027907,234.812751,AAPL,-0.008550,0.973891,-0.026109,-0.026109,25.692969,0.978424,0.002889
2010-06-28,AMZN,-0.026446,0.963127,-0.036873,-0.036873,117.800005,GOOG,-0.001269,0.970859,-0.029141,-0.029141,234.514685,AAPL,0.005999,0.979733,-0.020267,-0.020267,25.847112,0.971240,-0.007343


In [32]:
## performing similar process as in question 1 to compute the MTD and YTD returns of the portolio
portfolio_YTD = portfolio.groupby(pd.TimeGrouper('A'))

In [33]:
portfolio["Portfolio_YTD"] = portfolio_YTD['Portfolio_Cumulative'].transform(lambda x: x/x.iloc[0]-1.0)

In [34]:
portfolio_MTD = portfolio.groupby(pd.TimeGrouper('BM'))

In [35]:
portfolio["Portfolio_MTD"] = portfolio_MTD['Portfolio_Cumulative'].transform(lambda x: x/x.iloc[0]-1.0)

In [36]:
portfolio.head()

,AMZN_TICKER,AMZN_DAILY_RETURN,AMZN_Cumulative,AMZN_YTD,AMZN_MTD,AMZN_Daily Close Price,GOOG_TICKER,GOOG_DAILY_RETURN,GOOG_Cumulative,GOOG_YTD,...,AAPL_TICKER,AAPL_DAILY_RETURN,AAPL_Cumulative,AAPL_YTD,AAPL_MTD,AAPL_Daily Close Price,Portfolio_Cumulative,Portfolio_Daily_returns,Portfolio_YTD,Portfolio_MTD
DATE,,,,,,,,,,,,,,,,,,,,,
2010-06-22,AMZN,NaN,1.000000,0.000000,0.000000,122.310000,GOOG,NaN,1.000000,0.000000,...,AAPL,NaN,1.000000,0.000000,0.000000,26.381780,1.000000,NaN,0.000000,0.000000
2010-06-23,AMZN,-0.007031,0.992969,-0.007031,-0.007031,121.449999,GOOG,-0.008638,0.991362,-0.008638,...,AAPL,-0.010517,0.989483,-0.010517,-0.010517,26.104323,0.991271,-0.008729,-0.008729,-0.008729
2010-06-24,AMZN,-0.025690,0.967460,-0.032540,-0.032540,118.330004,GOOG,-0.014418,0.977069,-0.022931,...,AAPL,-0.007270,0.982289,-0.017711,-0.017711,25.914540,0.975606,-0.015803,-0.024394,-0.024394
2010-06-25,AMZN,0.022564,0.989290,-0.010710,-0.010710,121.000002,GOOG,-0.005094,0.972093,-0.027907,...,AAPL,-0.008550,0.973891,-0.026109,-0.026109,25.692969,0.978424,0.002889,-0.021576,-0.021576
2010-06-28,AMZN,-0.026446,0.963127,-0.036873,-0.036873,117.800005,GOOG,-0.001269,0.970859,-0.029141,...,AAPL,0.005999,0.979733,-0.020267,-0.020267,25.847112,0.971240,-0.007343,-0.028760,-0.028760


In [37]:
portfolio_final = portfolio[['Portfolio_Cumulative','Portfolio_Daily_returns','Portfolio_YTD','Portfolio_MTD']]

In [38]:
portfolio_final.head()

,Portfolio_Cumulative,Portfolio_Daily_returns,Portfolio_YTD,Portfolio_MTD
DATE,,,,
2010-06-22,1.000000,NaN,0.000000,0.000000
2010-06-23,0.991271,-0.008729,-0.008729,-0.008729
2010-06-24,0.975606,-0.015803,-0.024394,-0.024394
2010-06-25,0.978424,0.002889,-0.021576,-0.021576
2010-06-28,0.971240,-0.007343,-0.028760,-0.028760


## Thank you!